In [3]:
from core import initialize_bodies
import numpy as np

In [2]:
# initialize bodies with random masses, positions, and velocities over set range
# bodies[n] = Body(mass, position [x, y], velocity [vx, vy])
bodies = initialize_bodies(5, mass_range = [1, 10], position_range = [-25, 25], velocity_range = [-5, 5])

# printing sample values
print(bodies[0].mass)
print(bodies[0].position)
print(bodies[0].velocity)
print(len(bodies))

6.720859090967878
[-19.66146369   1.84198915]
[3.93846504 1.59208957]
5
